In [1]:
import glob
import json
from datetime import datetime
import pandas as pd

In [2]:
# Path pattern to locate all JSON files across snapshots
file_pattern = "C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset/snapshot_*/*_pr_sharings.json"

# Initialize an empty list to store all JSON data
all_data = []

# Loop through all matching file paths and load each file
for path in glob.glob(file_pattern):
    print(path)
    with open(path, "r") as file:
        data = json.load(file)
        all_data.append(data)

# all_data now contains data from all JSON files
print(f"Total files read: {len(all_data)}")

C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230727\20230727_195927_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230803\20230803_093947_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230810\20230810_123110_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230817\20230817_125147_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230824\20230824_100450_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230831\20230831_060603_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230907\20230907_091631_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Dataset\snapshot_20230914\20230914_074826_pr_sharings.json
C:/Users/Dell/Desktop/AI for Software Engineering/Project/Datase

In [3]:
# Initialize a list for rows that will form the DataFrame
rows = []

# Define the pattern analysis function
def analyze_prompt_structure(data):
    detected_patterns = []
    
    persona_keywords = ["you are", "act as", "pretend to be", "pretend you are"]
    recipe_keywords = ["step-by-step", "recipe", "guide"]
    template_keywords = ["template", "formatting"]
    automator_keywords = ["script", "code", "executable"]
    simple_instruction_keywords = ["explain", "describe", "list", "tell me", "give me"]
    context_instruction_keywords = ["based on", "with this information"]
    question_keywords = ["what", "where", "when", "who", "why"]

    def contains_keywords(text, keywords):
        return any(keyword in text.lower() for keyword in keywords)

    for source in data.get("Sources", []):
        body = source.get("Body", "")
        created_at = source.get("CreatedAt")
        closed_at = source.get("ClosedAt")
        state = "Closed" if closed_at is not None else "Open"

        time_lapsed = None
        if created_at and closed_at:
            created_at_dt = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")
            closed_at_dt = datetime.strptime(closed_at, "%Y-%m-%dT%H:%M:%SZ")
            time_lapsed = closed_at_dt - created_at_dt

        body_patterns = []

        # Check for patterns in the body text
        if contains_keywords(body, persona_keywords):
            body_patterns.append("Persona Pattern")
        if contains_keywords(body, recipe_keywords):
            body_patterns.append("Recipe Pattern")
        if contains_keywords(body, template_keywords):
            body_patterns.append("Template Pattern")
        if contains_keywords(body, automator_keywords):
            body_patterns.append("Output Automator Pattern")
        if contains_keywords(body, simple_instruction_keywords):
            body_patterns.append("Simple Instruction Pattern")
        if contains_keywords(body, context_instruction_keywords):
            body_patterns.append("Context and Instruction Pattern")
        if contains_keywords(body, question_keywords):
            body_patterns.append("Question Pattern")

        conversations = source.get("ChatgptSharing", [])
        for item in conversations:
            number_of_prompts = item.get("NumberOfPrompts", "N/A")
            conversation_url = item.get("URL", "N/A")

            for pattern in body_patterns:
                detected_patterns.append((
                    source['Number'],
                    pattern,
                    state,
                    time_lapsed,
                    number_of_prompts,
                    conversation_url
                ))

    return detected_patterns

In [4]:
# Loop through each dictionary in all_data
for data_dict in all_data:
    # Check if 'Sources' key exists in each dictionary
    if 'Sources' in data_dict:
        for item in data_dict['Sources']:
            if 'ChatgptSharing' in item:
                for sharing in item['ChatgptSharing']:
                    if 'Conversations' in sharing:
                        for conversation in sharing['Conversations']:
                            # Adding the fields to row
                            row = {
                                "Type": item.get('Type'),
                                "URL": item.get('URL'),
                                "Author": item.get('Author'),
                                "RepoName": item.get('RepoName'),
                                "RepoLanguage": item.get('RepoLanguage'),
                                "Number": item.get('Number'),
                                "Title": item.get('Title'),
                                "Body": item.get('Body'),
                                "MergedAt": item.get('MergedAt'),
                                "UpdatedAt": item.get('UpdatedAt'),
                                "State": item.get('State'),
                                "Additions": item.get('Additions'),
                                "Deletions": item.get('Deletions'),
                                "ChangedFiles": item.get('ChangedFiles'),
                                "CommitsTotalCount": item.get('CommitsTotalCount'),
                                "CSharing_URL": sharing.get('URL'),
                                "CSharing_Status": sharing.get('Status'),
                                "CSharing_DateOfConversation": sharing.get('DateOfConversation'),
                                "CSharing_Title": sharing.get('Title'),
                                "CSharing_NumberOfPrompts": sharing.get('NumberOfPrompts'),
                                "CSharing_TokensOfPrompts": sharing.get('TokensOfPrompts'),
                                "CSharing_TokensOfAnswers": sharing.get('TokensOfAnswers'),
                                "Conversation_Prompt": conversation.get('Prompt'),
                                "Conversation_Answer": conversation.get('Answer')
                            }
                            rows.append(row)

# Create DataFrame from rows
df = pd.DataFrame(rows)

In [5]:
# Detect patterns for each source and add to the DataFrame
detected_patterns = analyze_prompt_structure(data_dict)  # Use the last data_dict for patterns

In [6]:
# Prepare a dictionary to store detected patterns for easy lookup
pattern_dict = {}
for number, pattern, state, time_lapsed, number_of_prompts, conversation_url in detected_patterns:
    if number not in pattern_dict:
        pattern_dict[number] = []
    pattern_dict[number].append(pattern)

# Add a new column for detected patterns
df['Detected Patterns'] = df['Number'].map(lambda x: ', '.join(pattern_dict.get(x, [])))

In [7]:
df.info

<bound method DataFrame.info of               Type                                                URL  \
0     pull request            https://github.com/labdao/plex/pull/469   
1     pull request            https://github.com/labdao/plex/pull/469   
2     pull request            https://github.com/labdao/plex/pull/469   
3     pull request            https://github.com/labdao/plex/pull/469   
4     pull request            https://github.com/labdao/plex/pull/469   
...            ...                                                ...   
7971  pull request          https://github.com/openai/evals/pull/1083   
7972  pull request  https://github.com/VOICEVOX/voicevox_core/pull...   
7973  pull request       https://github.com/MatrixAI/Polykey/pull/552   
7974  pull request  https://github.com/VOICEVOX/voicevox_core/pull...   
7975  pull request     https://github.com/VOICEVOX/voicevox/pull/1374   

              Author                RepoName RepoLanguage  Number  \
0          AdamGoyer  

In [9]:
# Save the DataFrame to a CSV file
df.to_csv('pr_sharing_visualize_output_with_patterns.csv', index=False)